In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load pre-trained model
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Fungsi untuk merekomendasikan produk yang mirip berdasarkan nama produk yang diklik
def recommend_similar_products(clicked_product_name, product_data, top_k=5):
    # Ubah nama produk yang diklik menjadi huruf kecil
    clicked_product_name = clicked_product_name.lower()

    # Dapatkan representasi embedding untuk nama produk yang diklik
    clicked_product_embedding = model.encode([clicked_product_name], convert_to_tensor=True)

    # Ubah semua nama produk dalam data menjadi huruf kecil
    product_names = [name.lower() for name in product_data['Name'].tolist()]

    # Dapatkan representasi embedding untuk semua nama produk
    product_embeddings = model.encode(product_names, convert_to_tensor=True)

    # Hitung similarity antara nama produk yang diklik dengan semua nama produk lainnya
    similarity_scores = util.pytorch_cos_sim(clicked_product_embedding, product_embeddings)[0]

    # Urutkan produk berdasarkan similarity score
    similar_products_indices = similarity_scores.argsort(descending=True)

    # Ambil nama produk yang mirip berdasarkan embedding similarity
    similar_products_by_embedding = [product_names[i] for i in similar_products_indices[:top_k]]

    # Cari nama produk yang mengandung substring yang dicari
    similar_products_by_substring = [name for name in product_names if clicked_product_name in name]

    # Gabungkan hasil dari kedua pendekatan
    combined_results = similar_products_by_substring + [name for name in similar_products_by_embedding if name not in similar_products_by_substring]

    # Batasi hasil ke top_k
    final_recommendations = combined_results[:top_k]

    return final_recommendations

# Contoh penggunaan:
if __name__ == "__main__":
    # Baca dataset produk
    df = pd.read_csv('products_final.csv', delimiter=';')

    # Klik pada sebuah produk (misalnya, 'Coffee Arabica')
    clicked_product_name = 'floor'

    # Dapatkan rekomendasi produk yang mirip
    recommendations = recommend_similar_products(clicked_product_name, df, top_k=10)

    # Tampilkan rekomendasi
    print("Rekomendasi produk yang mirip dengan", clicked_product_name, ":")
    for product in recommendations:
        print(product)

Rekomendasi produk yang mirip dengan floor :
wood flooring
wood flooring
wood flooring
wood flooring
wood flooring
wood flooring
wood flooring
wood flooring
wood flooring
wood flooring
